# Lesson 1 : scaler and count

In this lesson, we'll work with a scaler (devices that counts pulses emitted from one or more pulse-emitting ***detector*** electronics) and use this to make a first lesson in using [Bluesky](http://nsls-ii.github.io/bluesky/) and [related tools](http://nsls-ii.github.io/).

First, we'll show how to start a Jupyter notebook.  Next, we'll connect with an EPICS scaler (using [ophyd](http://nsls-ii.github.io/ophyd/)), and then use the Bluesky software to count from the scaler.


**note**:  This tutorial expects to find an EPICS IOC on the local network configured as a synApps [`xxx`](https://github.com/epics-modules/xxx) IOC with prefix `sky:`.  A docker container is available to provide this IOC.  See this URL for instructions:  https://github.com/prjemian/epics-docker/blob/master/n3_synApps/README.md


## Starting this session in a Jupyter notebook

This session was started from the linux command line:

```
jemian@otz ~ $ source /APSshare/anaconda3/Bluesky/bin/activate 
(base) jemian@otz ~ $ jupyter-notebook 
```

This command produced the following console output and then started my default web browser with a one-time-token-authenticated connection to the Jupyter Notebook server (still running in the console):

```
[I 15:16:57.546 NotebookApp] Serving notebooks from local directory: /home/oxygen18/JEMIAN
[I 15:16:57.546 NotebookApp] 0 active kernels
[I 15:16:57.546 NotebookApp] The Jupyter Notebook is running at:
[I 15:16:57.546 NotebookApp] http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:16:57.546 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 15:16:57.547 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://localhost:8888/?token=e6a7584762c731a7c64f8f71246b3e616d779f7b4852c9d9
[I 15:17:00.863 NotebookApp] Accepting one-time-token-authenticated connection from ::1
```

Next, found the **New** drop-down menu button (top right, below the **Lougout** button) and selected *Python 3* to start a new notebook page using a Python 3 shell (the only kind available here).

Finally, from the **File** menu in the jupyter notebook (in the browser), selected *Rename ...* to save the *Untitled* notebook with the name *lesson1* (default extension is `.ipynb`).

## Connect an EPICS scaler

I have a synApps (v6.1) *XXX*-style IOC with the prefix `sky:`.  It has a [scaler](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/std/R3-4-1/documentation/scalerRecord.html), 16 soft channel [motor](https://github.com/epics-modules/motor)s, and some other support we'll ignore in lesson 1.

The scaler is `sky:scaler1`.  We'll connect to that first.  To make the connection, we need to import the [`ScalerCH`](http://nsls-ii.github.io/ophyd/builtin-devices.html#epicsscaler) device from the [`ophyd.scaler`](http://nsls-ii.github.io/ophyd/) library.

In [1]:
from ophyd.scaler import ScalerCH

Now we can create the scaler object we'll use as a detector.

In [2]:
scaler = ScalerCH("sky:scaler1", name="scaler")

In a script or program, we should wait for that to connect with EPICS.

In [3]:
scaler.wait_for_connection()

Let's test that connection by asking the scaler to read its values from EPICS.

In [4]:
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1588881330.334138}),
             ('I0', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('scint', {'value': 10.0, 'timestamp': 1588881330.334138}),
             ('diode', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('roi1', {'value': 7.0, 'timestamp': 1588881330.334138}),
             ('I0Mon', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('ROI1', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('ROI2', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1588881330.334138})])

By default, the ophyd `ScalerCH` object shows data for only the channels with names defined in EPICS.  Except for one channel (probably a bug in the `ScalerCH` code).  To fix that, we'll load another support library.  (Loading additional support as we need it will be a common theme in these lessons.  Rather than loading all the libraries first, as is common in python code files, we'll load support code as the need arises.)

The new support code is `use_EPICS_scaler_channels` from [`apstools.devices`](http://apstools.readthedocs.io/en/latest/source_code/devices.html#apstools.devices.use_EPICS_scaler_channels).

Our scaler only has a few channels in use (that is, channels where the name of the channel has been specified in the GUI screen).  Let's focus on just those channels.

To read just the *named* channels, we call the scaler's `select_channels()` method and name the scaler channels we wish to see.  To see all channels with names defined in EPICS, use `None` as shown:

In [5]:
scaler.select_channels(None)
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1588881330.334138}),
             ('I0', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('scint', {'value': 10.0, 'timestamp': 1588881330.334138}),
             ('diode', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('roi1', {'value': 7.0, 'timestamp': 1588881330.334138}),
             ('I0Mon', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('ROI1', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('ROI2', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1588881330.334138})])

If you want to change the name fields on any of the scaler channels from the command line, follow this example.

In [6]:
scaler.channels.chan04.chname.put("scint")
scaler.channels.chan07.chname.put("roi1")

Then, update the `scaler` object for these channels.

In [7]:
scaler.select_channels(None)
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1588881330.334138}),
             ('I0', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('scint', {'value': 10.0, 'timestamp': 1588881330.334138}),
             ('diode', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('roi1', {'value': 7.0, 'timestamp': 1588881330.334138}),
             ('I0Mon', {'value': 9.0, 'timestamp': 1588881330.334138}),
             ('ROI1', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('ROI2', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1588881330.334138})])

## Get, then Set the count time on the scaler

The preset counting time (`.TP`) is one of the many fields of the EPICS scaler record.  It is available as `scaler.preset_time` in this scaler object.  We can inspect this:

In [8]:
scaler.preset_time

EpicsSignal(read_pv='sky:scaler1.TP', name='scaler_preset_time', parent='scaler', timestamp=1588881330.334138, tolerance=0.001, auto_monitor=False, string=False, write_pv='sky:scaler1.TP', limits=False, put_complete=False)

Since `scaler.preset_time` is an [`EpicsSignal`](https://github.com/NSLS-II/ophyd/blob/master/ophyd/signal.py#L647), ([Signal](http://nsls-ii.github.io/ophyd/signals.html) is a fundamental ophyd structure), we print its `.value`.

In [9]:
scaler.preset_time.get()

1.5

Set the scaler to count for 1.5 seconds once it is told to count.  We'll count it next.

In [10]:
scaler.preset_time.put(1.5)

Note, to tell the scaler to count, we put a 1 to its `.CNT` field.

In [11]:
scaler.count.put(1)

Now, read the scaler again.  This scaler is not very interesting since it is not connected to any hardware.  But, the timestamps have changed.

In [12]:
scaler.read()

OrderedDict([('clock', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('I0', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('scint', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('diode', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('roi1', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('I0Mon', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('ROI1', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('ROI2', {'value': 0.0, 'timestamp': 1588881330.334138}),
             ('scaler_time', {'value': 0.0, 'timestamp': 1588881330.334138})])

## Use Bluesky to count the scaler

So far, we have used ophyd to talk with the EPICS scaler.  Let's start to use Bluesky.  We need to load support to start the Bluesky [*RunEngine*](http://nsls-ii.github.io/bluesky/tutorial.html#the-runengine).  For now, we'll use the most basic configuration (does not save data anywhere).  

The job of the RunEngine is to process command messages (our data acquisition commands) and to output documents (the data to be acquired).  Initially, we'll use predefined data acquisition sequences (called [*plans*](http://nsls-ii.github.io/bluesky/plans.html)) and ignore the [documents](http://nsls-ii.github.io/bluesky/documents.html).

In [13]:
from bluesky import RunEngine
RE = RunEngine({})

Next, we'll load a library with many [predefined plans](http://nsls-ii.github.io/bluesky/plans.html#summary).  The first plan we want to use is [`count()`](http://nsls-ii.github.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  By the way, we'll use a python feature to rename `bluesky.plans` to the shorter `bp` since it will be used a lot.

In [14]:
import bluesky.plans as bp

Now we can count the `scaler` by submitting the `bp.count` plan to the RunEngine instance `RE`.  The `bp.count()` plan takes one argument, a *list* of countable objects (just in case you want to count more than one detector at the same time).  Our list has only one object: `scaler`

In [15]:
RE(bp.count([scaler]))

('1e31385f-80bc-4d2e-9d81-a958290283c8',)

The final result from submitting a plan to the RunEngine is an identifier of the sequence of documents.  Since we did not capture that sequence, we can't view it or access it in any way.  Let's *fix* that next.

### Report about the documents emitted from the RunEngine

To display information about the documents emitted from the RunEngine, we need to make a function that will receive the documents.  

In Bluesky terms, this type of function is a *callback*.  It takes two arguments.  The first argument is a `str` that tells what kind of document is coming, the second is a python dictionary with the document's contents.  We'll start by printing summary information.  We then submit the plan again with the name of our callback function as a second argument (not to `bp.count` but as a second argument to `RE()`).

In [16]:
def myCallbackBrief(key, doc):
    print(key, len(doc))

In [17]:
RE(bp.count([scaler]), myCallbackBrief)

start 11
descriptor 8
event 7
stop 6


('139ccaa3-6605-415f-9fb9-b0b6da00fbb7',)

So, there were four documents.  See http://nsls-ii.github.io/bluesky/documents.html for the details of each.  Let's extend our callback to print the details of each.

In [18]:
def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")

In [19]:
RE(bp.count([scaler]), myCallback)

start 11
	 uid 2d96c5a0-fc6e-4d3c-80bd-ad5071a71082
	 time 1588881355.1349988
	 versions {'ophyd': '1.5.0', 'bluesky': '1.6.0'}
	 scan_id 3
	 plan_type generator
	 plan_name count
	 detectors ['scaler']
	 num_points 1
	 num_intervals 0
	 plan_args {'detectors': ["ScalerCH(prefix='sky:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan04', 'channels.chan04.s', 'channels.chan05', 'channels.chan05.s', 'channels.chan07', 'channels.chan07.s', 'channels.chan08', 'channels.chan08.s', 'channels.chan10', 'channels.chan10.s', 'channels.chan11', 'channels.chan11.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 

('2d96c5a0-fc6e-4d3c-80bd-ad5071a71082',)

With the details, we see more of what is happening.  

The first document is a `start` and it says what is happening and how it will be identified.  The identifier (named `uid`) is a [UUID](https://docs.python.org/3.6/library/uuid.html) (specifically, a [uuid4](https://docs.python.org/3.6/library/uuid.html#uuid.uuid4)).  This looks tedious and random.  The one good thing is that these are unique and often can be referred to by the first few characters (6-8 are enough to be *probably* unique).  The `uid` from the `start` document is what the RunEngine reports when the scan stops.

The second document is a `descriptor` and it says what, exactly, will be measured.  It also includes a reference to `start`'s `uid`.  We can follow the chain back to `start` this way.

The third document is an `event` and it provides the data reading when the scaler was counted.  The data is read according to the `descriptor` document (just actual values for the *named* channels).

The last document is a `stop` document that describes how things ended.  It also says there was only one stream of event documents named *primary*.

Let's repeat that plan, this time asking to `count` the scaler three times.  This is the `num=3` addition to the call to `bp.count()`.  What's tricky now is keeping track of the parentheses and square braces.

In [20]:
RE(bp.count([scaler], num=3), myCallback)

start 11
	 uid 5807d3f5-5b23-4c04-95c1-9dd516b6bbc5
	 time 1588881356.8956878
	 versions {'ophyd': '1.5.0', 'bluesky': '1.6.0'}
	 scan_id 4
	 plan_type generator
	 plan_name count
	 detectors ['scaler']
	 num_points 3
	 num_intervals 2
	 plan_args {'detectors': ["ScalerCH(prefix='sky:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan04', 'channels.chan04.s', 'channels.chan05', 'channels.chan05.s', 'channels.chan07', 'channels.chan07.s', 'channels.chan08', 'channels.chan08.s', 'channels.chan10', 'channels.chan10.s', 'channels.chan11', 'channels.chan11.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 

event 7
	 descriptor 7f4c9415-97a5-4358-9407-06ee45ffea4a
	 time 1588881360.1128552
	 data {'clock': 16000000.0, 'I0': 9.0, 'scint': 8.0, 'diode': 5.0, 'roi1': 7.0, 'I0Mon': 7.0, 'ROI1': 0.0, 'ROI2': 0.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1588881360.10127, 'I0': 1588881360.10127, 'scint': 1588881360.10127, 'diode': 1588881360.10127, 'roi1': 1588881360.10127, 'I0Mon': 1588881360.10127, 'ROI1': 1588881360.10127, 'ROI2': 1588881360.10127, 'scaler_time': 1588881360.10127}
	 seq_num 2
	 uid 8a91159d-d376-40ad-83d1-7d132f52ee88
	 filled {}
~~~~~~~~~~~~~~~~~
event 7
	 descriptor 7f4c9415-97a5-4358-9407-06ee45ffea4a
	 time 1588881361.7444198
	 data {'clock': 16000000.0, 'I0': 6.0, 'scint': 6.0, 'diode': 8.0, 'roi1': 9.0, 'I0Mon': 9.0, 'ROI1': 0.0, 'ROI2': 0.0, 'scaler_time': 1.6}
	 timestamps {'clock': 1588881361.740096, 'I0': 1588881361.740096, 'scint': 1588881361.740096, 'diode': 1588881361.740096, 'roi1': 1588881361.740096, 'I0Mon': 1588881361.740096, 'ROI1': 1588881361.740096, 'RO

('5807d3f5-5b23-4c04-95c1-9dd516b6bbc5',)

Mostly the same but now there are three `event` documents.

# Summary

We'll show this code as a python program:

```
#!/usr/bin/env python

"lesson 1: EpicsScaler"

from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from apstools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")


RE = RunEngine({})

scaler = ScalerCH("sky:scaler1", name="scaler")
scaler.preset_time.put(1.5)
print(scaler.preset_time.value)

scaler.channels.chan04.chname.put("scint")
scaler.channels.chan07.chname.put("roi1")

scaler.match_names()
use_EPICS_scaler_channels(scaler)
print(scaler.read())
print(RE(bp.count([scaler], num=3), myCallback))
```


## Quit the jupyter notebook

* Save the "lesson1" page if you wish.
* Press the **Logout** button in the upper right corner of "lesson1" and close the page.
* Press the **Logout** button in the upper right corner of "Home" and close the page.
* Type Control-C (^C) twice in the jupyter-notebook console to kill the server.